# 이 예제는 그냥 인코더만 있는 예제. 별 의미는 없는 듯

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.chdir(r"c:\projects\PY_MLE")

import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from gensim.models.doc2vec import TaggedDocument

import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Input, LSTM, Dense, GlobalAveragePooling1D, SimpleRNN, Embedding, Masking, Bidirectional, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Concatenate
import sentencepiece as spm
import urllib.request
import csv
import unicodedata

In [44]:
vocab_size = 10000
(x_train, y_train),(x_test, y_test) = imdb.load_data(num_words=vocab_size)

In [45]:
print("최대 길이: {}".format(max(len(x) for x in x_train)))
print("평균 길이: {}".format(sum(map(len, x_train))/len(x_train)))

최대 길이: 2494
평균 길이: 238.71364


In [46]:
# 패팅을 수행
max_len = 500
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

In [73]:
sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = Embedding(vocab_size, 128, input_length=max_len, mask_zero=True)(sequence_input)
lstm = Bidirectional(LSTM(64, dropout=0.5, return_sequences=True))(embedded_sequences)

# return_state를 했기 때문에 forwar_h...에는 마지막값 저장되어 있을 뜻, lstm모든 값이 저장되어 있을 듯
lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional(LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)

In [75]:
# state_h는 인코더의 마지막 은닉상택값이다. 왜 정방향의 마지막값과 역방향의 시작값을 연결한게 마지막시점의 은닉상태값인는 2장 211쪽에 잘 나와있다
# 만약에 정방향 마지막 시점의 값과 동일 시점의 역바향값을 연결해서 그 시점의 은닉상태값으로 할 경우
# 역방향은 모든 시점의 은닉상태값을 참조한게 아니기 때문이다. 결국 이런 결론에 이룰 수 있다. 정뱡향 어느 시점의 은닉상태값은
# 그 시점의 정방향 값과 역방향 시작점 값을 연결하는 것이다. 역방향 시작점은 무조건 시점 1이 겠지
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)


(None, 500, 128) (None, 64) (None, 64) (None, 64) (None, 64)


In [76]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = Dense(units)
    self.W2 = Dense(units)
    self.V = Dense(1)

  def call(self, values, query): # 단, key와 value는 같음
    # query shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # score 계산을 위해 뒤에서 할 덧셈을 위해서 차원을 변경해줍니다.
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights


In [80]:
attention = BahdanauAttention(64)
# lstm은 각 시점의 모든 은닉상태, state_h는 마지막 시점의 은닉상택값이다.

context_vector, attention_weight = attention(lstm, state_h)
dense1 = Dense(20, activation="relu")(context_vector)
dropout = Dropout(0.5)(dense1)
output = Dense(1, activation="sigmoid")(dropout)
model = Model(inputs=sequence_input, outputs=output)

In [83]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy' ])

In [ ]:
history = model.fit(x_train, y_train, epochs = 3, batch_size = 256, validation_data=(x_test, y_test), verbose=1)

Epoch 1/3


98/98 [==============================] - 1181s 12s/step - loss: 0.4732 - accuracy: 0.7655 - val_loss: 0.2842 - val_accuracy: 0.8806
Epoch 2/3
38/98 [==========>...................] - ETA: 10:43 - loss: 0.2306 - accuracy: 0.9197